In [19]:
# Training loop:
# 1. Create torch dataset from the data containing the query, positive, negative example.
# 2. Create a dataloader from the dataset.
# 3. Run a single batch through the model.

In [20]:
!poetry add pandas

The following packages are already present in the pyproject.toml and will be skipped:

  • pandas

If you want to update it to the latest compatible version, you can use `poetry update package`.
If you prefer to upgrade it to the latest available version, you can use `poetry add package@latest`.

Nothing to add.


In [21]:
from torch.utils.data import DataLoader, Dataset
import pandas as pd

In [342]:
## Load data
train = pd.read_parquet("train.parquet")
test = pd.read_parquet("test.parquet")
validate = pd.read_parquet("validate.parquet")

In [23]:
print(train.shape, test.shape, validate.shape)

(82326, 6) (9650, 6) (10047, 6)


In [24]:
full_dataset = pd.concat([train, test, validate])

In [241]:
full_dataset[full_dataset["query_id"].isin([19699, 19701])]

,answers,passages,query,query_id,query_type,wellFormedAnswers
0,[Results-Based Accountability is a disciplined...,"{'is_selected': [0, 0, 0, 0, 0, 1, 0, 0, 0, 0]...",what is rba,19699,description,[]
2,[20-25 minutes],"{'is_selected': [0, 0, 0, 0, 1, 0, 0, 0, 0, 0]...",how long do you need for sydney and surroundin...,19701,numeric,[]


In [ ]:
# Save the Query IDs for each dataset to use on later.

In [25]:
total_size = train.shape[0] + test.shape[0] + validate.shape[0]
print(total_size)
# Rerun this code only if you want to generate new indices.
import numpy as np
import pickle

# np.random.seed(seed=999999)
# indices = np.random.randint(0, total_size, total_size)
# train_indices = indices[0:train.shape[0]]
# test_indices = indices[train.shape[0]:train.shape[0]+test.shape[0]]
# validate_indices = indices[train.shape[0]+test.shape[0]:]
# all_indices = {'train': train_indices.tolist(), 'test': test_indices.tolist(), 'validate': validate_indices.tolist()}
# print(f"Length of all indices: {sum(len(v) for k, v in all_indices.items())}")
# # write these to a pickle file

# with open('indices.pkl', 'wb') as f:
#     pickle.dump(all_indices, f)

# create train, test, validation datasets based on query_ids
# query_ids = full_dataset['query_id']
# train_query_ids = query_ids[0:train.shape[0]]
# test_query_ids = query_ids[train.shape[0]:train.shape[0]+test.shape[0]]
# validate_query_ids = query_ids[train.shape[0]+test.shape[0]:]
# print(train_query_ids.shape, test_query_ids.shape, validate_query_ids.shape)
# all_query_ids  = {'train': train_query_ids.tolist(), 'test': test_query_ids.tolist(), 'validate': validate_query_ids.tolist()}
# # write these to a pickle file

# with open('query_ids.pkl', 'wb') as f:
#     pickle.dump(all_query_ids, f)

102023


In [351]:
# Unpickle and test
with open("indices.pkl", "rb") as f:
    all_indices = pickle.load(f)

    print(f"{len(all_indices['train'])} training indices {all_indices['train'][0:10]}")
    print(f"{len(all_indices['test'])} test indices {all_indices['test'][0:10]}")
    print(
        f"{len(all_indices['validate'])} validate indices {all_indices['validate'][0:10]}"
    )

with open("query_ids.pkl", "rb") as f:
    all_query_ids = pickle.load(f)
    print(
        f"{len(all_query_ids['train'])} training query ids {all_query_ids['train'][0:10]}"
    )
    print(f"{len(all_query_ids['test'])} test query ids {all_query_ids['test'][0:10]}")
    print(
        f"{len(all_query_ids['validate'])} validate query ids {all_query_ids['validate'][0:10]}"
    )

82326 training indices [36233, 13668, 19591, 66699, 34957, 34882, 23273, 27926, 90079, 68436]
9650 test indices [27928, 5317, 97860, 83225, 90861, 42316, 42625, 98527, 74133, 95026]
10047 validate indices [94011, 72401, 60227, 70263, 35765, 47026, 31472, 3385, 21289, 545]
82326 training query ids [19699, 19700, 19701, 19702, 19703, 19704, 19705, 19706, 19707, 19708]
9650 test query ids [0, 1, 2, 3, 4, 5, 6, 7, 8, 9]
10047 validate query ids [9652, 9653, 9654, 9655, 9656, 9657, 9658, 9659, 9660, 9661]


In [361]:
# train[['passages', 'query']].iloc[0]
train_dataset = full_dataset[full_dataset.query_id.isin(all_query_ids["train"])]
test_dataset = full_dataset[full_dataset.query_id.isin(all_query_ids["test"])]
validate_dataset = full_dataset[full_dataset.query_id.isin(all_query_ids["validate"])]

train_dataset.head()["query"].iloc[0]

'what is rba'

In [362]:
print(f"{np.all(train.query_id.value_counts() == 1)}")
print(f"{np.all(train_dataset.query_id.value_counts() == 1)}")
assert np.all(train.query_id.value_counts() == 1)
assert np.all(train_dataset.query_id.value_counts() == 1)
assert np.all(validate.query_id.value_counts() == 1)

True
True


In [238]:
full_dataset.head()

,answers,passages,query,query_id,query_type,wellFormedAnswers
0,[Results-Based Accountability is a disciplined...,"{'is_selected': [0, 0, 0, 0, 0, 1, 0, 0, 0, 0]...",what is rba,19699,description,[]
1,[Yes],"{'is_selected': [0, 1, 0, 0, 0, 0, 0], 'passag...",was ronald reagan a democrat,19700,description,[]
2,[20-25 minutes],"{'is_selected': [0, 0, 0, 0, 1, 0, 0, 0, 0, 0]...",how long do you need for sydney and surroundin...,19701,numeric,[]
3,[$11 to $22 per square foot],"{'is_selected': [0, 0, 0, 0, 0, 0, 0, 0, 1], '...",price to install tile in shower,19702,numeric,[]
4,[Due to symptoms in the body],"{'is_selected': [0, 0, 1, 0, 0, 0, 0, 0], 'pas...",why conversion observed in body,19703,description,[]


In [ ]:
from utils.data_utils import create_lookups, add_hashed_urls

In [364]:
ids_to_urls, urls_to_ids, url_to_doctext_mapping = create_lookups(full_dataset)
add_hashed_urls(train_dataset, urls_to_ids)
add_hashed_urls(test_dataset, urls_to_ids)
add_hashed_urls(validate_dataset, urls_to_ids)

dataset shape: (102023, 6)
Total number of urls: 837729
Total number of unique urls: 456487
Total number of hashed urls: 456487
Total number of queries: 102023


100%|██████████| 82326/82326 [00:00<00:00, 197706.24it/s]
/var/folders/fk/dlmj8gqd4ms2jgbkv_33skth0000gn/T/ipykernel_77110/1830918971.py:40: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dataset.loc[:, "hashed_urls"] = dataset["passages"].progress_apply(
100%|██████████| 9650/9650 [00:00<00:00, 183872.95it/s]
/var/folders/fk/dlmj8gqd4ms2jgbkv_33skth0000gn/T/ipykernel_77110/1830918971.py:40: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dataset.loc[:, "hashed_urls"] = dataset["passages"].progress_apply(
100%

In [532]:
with open("url_to_doctext_mapping.pkl", "wb") as f:
    pickle.dump(url_to_doctext_mapping, f)

In [279]:
train["passages"].iloc[0]["url"]

array(['https://en.wikipedia.org/wiki/Reserve_Bank_of_Australia',
       'https://en.wikipedia.org/wiki/Reserve_Bank_of_Australia',
       'http://acronyms.thefreedictionary.com/RBA',
       'https://www.slimvapepen.com/rebuildable-atomizer-rba/',
       'http://rba-africa.com/about/what-is-rba/',
       'http://resultsleadership.org/what-is-results-based-accountability-rba/',
       'http://rba-africa.com/about/what-is-rba/',
       'http://searchsecurity.techtarget.com/definition/risk-based-authentication-RBA',
       'https://www.slimvapepen.com/rebuildable-atomizer-rba/',
       'http://www.rbaconsulting.com/'], dtype=object)

In [365]:
print(url_to_doctext_mapping["51456f6eae52e682e8cc2877123547df"])
print(urls_to_ids["http://rba-africa.com/about/what-is-rba/"])
print(ids_to_urls["c2f05ed2ec73e1b97799c70e4cdbed8f"])

For example, you could contribute $2,500 to a Roth IRA with one provider and $3,000 with a second provider. You cannot contribute $5,500 to the first Roth IRA and $5,500 to the second provider. Your total contribution limit is $5,500 for all accounts for the tax year. Individuals may contribute money to a Roth IRA from January 1 of the current tax year to April 15 of the following tax year. Contributions are made with post-tax dollars. You pay tax today so that in retirement no tax is charged to you in retirement.
c2f05ed2ec73e1b97799c70e4cdbed8f
http://rba-africa.com/about/what-is-rba/


In [ ]:
from tqdm import tqdm

tqdm.pandas()

In [366]:
master_url_id_set = np.array(
    list(set(ids_to_urls.keys()))
)  # Define or load your master_url_id_set here

# Triples

In [131]:
# Splitting the dataset for demonstration; in practice, adjust based on your dataset size
from concurrent.futures import ProcessPoolExecutor, as_completed
from functools import partial
import utils
import importlib

importlib.reload(utils)
importlib.reload(utils.data_utils)
from utils.data_utils import add_negative_samples

In [367]:
training_dataset_copy = train_dataset.copy()
add_negative_samples(training_dataset_copy, ids_to_urls)

/Users/sid/workspace/mlx-week3/.venv/lib/python3.11/site-packages/numpy/core/fromnumeric.py:59: FutureWarning: 'Series.swapaxes' is deprecated and will be removed in a future version. Please use 'Series.transpose' instead.
  return bound(*args, **kwds)
Processing Batches: 100%|██████████| 10/10 [11:02<00:00, 66.24s/it]  


In [368]:
assert np.all(training_dataset_copy.query_id.value_counts() == 1)

In [370]:
test_dataset_copy = test_dataset.copy()
add_negative_samples(test_dataset_copy, ids_to_urls)

/Users/sid/workspace/mlx-week3/.venv/lib/python3.11/site-packages/numpy/core/fromnumeric.py:59: FutureWarning: 'Series.swapaxes' is deprecated and will be removed in a future version. Please use 'Series.transpose' instead.
  return bound(*args, **kwds)
Processing Batches: 100%|██████████| 10/10 [01:18<00:00,  7.88s/it]


In [371]:
# Sanity check
assert np.all(test_dataset_copy.query_id.value_counts() == 1)

In [372]:
validate_dataset_copy = validate_dataset.copy()
add_negative_samples(validate_dataset_copy, ids_to_urls)

/Users/sid/workspace/mlx-week3/.venv/lib/python3.11/site-packages/numpy/core/fromnumeric.py:59: FutureWarning: 'Series.swapaxes' is deprecated and will be removed in a future version. Please use 'Series.transpose' instead.
  return bound(*args, **kwds)
Processing Batches: 100%|██████████| 10/10 [01:29<00:00,  8.98s/it]


In [373]:
# Sanity check
assert np.all(validate_dataset_copy.query_id.value_counts() == 1)

In [410]:
training_dataset_copy[triple_columns].head()

,query_id,query,hashed_urls,negative_sample_urls
0,19699,what is rba,"[58b2b8024507a2126d53e59a87820ad8, affa5fde381...","[a7509f1dc6a82e1058cac78d284eee49, 9673999d279..."
1,19700,was ronald reagan a democrat,"[a2d179699e02966bdf9faa65b992daab, f097b1f42e4...","[06aa259b817892b5290b4b037c376591, 79b11837191..."
2,19701,how long do you need for sydney and surroundin...,"[70185a71a8f71d8d1029a71419d5cabf, f8e326ab289...","[6c364273217b774565d18c413d7b0095, 4e96bb1cbd3..."
3,19702,price to install tile in shower,"[49abbb4ce58adb0395a9ca4e1846239c, 0a1266e71ec...","[75dc8659a62b8bff44831a44f18463aa, 70d696bab58..."
4,19703,why conversion observed in body,"[de25b567b657116e9617dabbf554ac44, ea0e0b397cf...","[bf10f48ebac0dd6ece976b659f25d061, 2606c7238d4..."


In [374]:
triple_columns = ["query_id", "query", "hashed_urls", "negative_sample_urls"]
training_dataset_copy[triple_columns].to_parquet(
    "training_dataset_triplets.parquet", engine="pyarrow"
)

In [375]:
test_dataset_copy[triple_columns].to_parquet(
    "test_dataset_triplets.parquet", engine="pyarrow"
)

In [376]:
validate_dataset_copy[triple_columns].to_parquet(
    "validate_dataset_triplets.parquet", engine="pyarrow"
)

In [377]:
# There is a unit test to test this in the tests/ dir.
s1 = list([len(i[1]["negative_sample_urls"]) for i in training_dataset_copy.iterrows()])
s2 = list([len(i[1]["hashed_urls"]) for i in training_dataset_copy.iterrows()])
print(len(s1))
print(len(s2))
print(s1[0:10])
print(s2[0:10])
assert s1 == s2

82326
82326
[7, 6, 6, 6, 7, 8, 4, 6, 7, 5]
[7, 6, 6, 6, 7, 8, 4, 6, 7, 5]


# Triples -> Embeddings


In [159]:
import sentencepiece as spm
from gensim.models import Word2Vec
import multiprocessing

In [160]:
sp = spm.SentencePieceProcessor()
sp.load("spm_AllTexts.model")

True

In [161]:
EMBEDDING_SIZE = 128

In [162]:
w2v_model = Word2Vec(
    min_count=20,
    window=10,
    vector_size=EMBEDDING_SIZE,
    sample=6e-5,
    alpha=0.03,
    min_alpha=0.0007,
    negative=20,
    workers=multiprocessing.cpu_count() - 1,
)

In [163]:
w2v_model = Word2Vec.load("word2vec.model")

### Convert the tripleID's to embeddings

In [499]:
import importlib

importlib.reload(utils)
importlib.reload(utils.data_utils)
from utils.data_utils import triples_to_embeddings

In [500]:
training_triples = triples_to_embeddings(
    training_dataset_copy[triple_columns],
    url_to_doctext_mapping,
    sp,
    w2v_model,
    embedding_size=EMBEDDING_SIZE,
)

  0%|          | 0/82326 [00:00<?, ?it/s]

There are 7 emebddings with [44, 70, 103, 146, 147, 60, 145]


ValueError: expected sequence of length 44 at dim 1 (got 70)

In [458]:
print(len(training_triples[0][2]))
print(len(training_triples[0][3]))

7
7


In [487]:
testing_triples = triples_to_embeddings(
    test_dataset_copy[triple_columns],
    url_to_doctext_mapping,
    sp,
    w2v_model,
    embedding_size=EMBEDDING_SIZE,
)

100%|██████████| 9650/9650 [02:43<00:00, 59.18it/s]


In [492]:
testing_triples[0].query_embedding

triple(query_id=0, query_embedding=tensor([[-3.8118e-01, -1.7516e+00,  1.1853e-01, -4.4650e+00, -5.0346e-01,
          1.0803e+00,  8.5209e-01, -1.2138e+00,  1.0042e+00, -3.7721e+00,
          6.2760e-01,  1.0943e+00,  3.9942e+00, -5.9516e+00, -1.6203e+00,
          1.4496e+00, -3.9064e-01,  1.7156e+00, -1.1008e+00, -1.8131e-01,
         -1.4462e+00,  8.8895e-01, -3.1144e+00, -3.4060e+00, -1.8267e+00,
          2.0124e-01, -7.8651e-01, -2.7603e+00,  3.0784e+00, -1.1899e+00,
         -3.9205e-01, -2.7277e+00, -2.7045e+00,  1.5575e+00, -3.1577e+00,
          2.8354e+00,  3.1483e-02, -1.8919e+00, -1.5236e+00,  3.5899e-01,
          3.4359e+00, -3.3176e-01,  1.9203e+00, -1.9457e+00,  2.2316e-01,
          3.1599e+00,  2.6357e-01, -2.9806e+00, -5.2368e+00, -4.7591e-01,
          1.7809e+00, -1.9811e+00, -1.1383e+00, -3.5465e+00, -2.2566e+00,
          3.6638e+00, -3.2768e+00,  3.7936e+00,  3.2374e-02,  9.3251e-01,
          1.9666e+00, -2.3154e+00, -1.9297e+00, -3.5151e-01, -1.5234e+00,
   

In [380]:
validation_triples = triples_to_embeddings(
    validate_dataset_copy[triple_columns],
    url_to_doctext_mapping,
    sp,
    w2v_model,
    embedding_size=EMBEDDING_SIZE,
)

100%|██████████| 10047/10047 [00:21<00:00, 471.36it/s]


In [412]:
# Pickle these to use for training
import pickle

with open("training_triples.pkl", "wb") as f:
    pickle.dump([tuple(i) for i in training_triples], f)
with open("testing_triples.pkl", "wb") as f:
    pickle.dump([tuple(i) for i in testing_triples], f)
with open("validation_triples.pkl", "wb") as f:
    pickle.dump([tuple(i) for i in validation_triples], f)

### Create Dataloaders

In [296]:
# Create a torch dataset from the triples
from torch.utils.data import Dataset, DataLoader

In [332]:
len(training_triples)

82326

In [338]:
training_dataset_copy.shape

(82326, 9)

In [350]:
# training_dataset_copy[(training_dataset_copy.query_id == 55966)]
np.all(train.query_id.value_counts().values == 1)
training_dataset_copy.query_id.value_counts()

query_id
62367    7
10515    7
16294    7
5458     6
38694    6
        ..
26998    1
13036    1
37958    1
52395    1
36237    1
Name: count, Length: 56417, dtype: int64

In [396]:
training_dataset_copy.head()

,answers,passages,query,query_id,query_type,wellFormedAnswers,hashed_urls,negative_sample_urls
0,[Results-Based Accountability is a disciplined...,"{'is_selected': [0, 0, 0, 0, 0, 1, 0, 0, 0, 0]...",what is rba,19699,description,[],"[58b2b8024507a2126d53e59a87820ad8, affa5fde381...","[a7509f1dc6a82e1058cac78d284eee49, 9673999d279..."
1,[Yes],"{'is_selected': [0, 1, 0, 0, 0, 0, 0], 'passag...",was ronald reagan a democrat,19700,description,[],"[a2d179699e02966bdf9faa65b992daab, f097b1f42e4...","[06aa259b817892b5290b4b037c376591, 79b11837191..."
2,[20-25 minutes],"{'is_selected': [0, 0, 0, 0, 1, 0, 0, 0, 0, 0]...",how long do you need for sydney and surroundin...,19701,numeric,[],"[70185a71a8f71d8d1029a71419d5cabf, f8e326ab289...","[6c364273217b774565d18c413d7b0095, 4e96bb1cbd3..."
3,[$11 to $22 per square foot],"{'is_selected': [0, 0, 0, 0, 0, 0, 0, 0, 1], '...",price to install tile in shower,19702,numeric,[],"[49abbb4ce58adb0395a9ca4e1846239c, 0a1266e71ec...","[75dc8659a62b8bff44831a44f18463aa, 70d696bab58..."
4,[Due to symptoms in the body],"{'is_selected': [0, 0, 1, 0, 0, 0, 0, 0], 'pas...",why conversion observed in body,19703,description,[],"[de25b567b657116e9617dabbf554ac44, ea0e0b397cf...","[bf10f48ebac0dd6ece976b659f25d061, 2606c7238d4..."


In [404]:
for i, training_triple in enumerate(training_triples[0:5]):
    print(f"{i}th triple with query_id: {training_triple[0]}")
    print(
        f"\t#Relevant Docs = {len(training_triple.relevant_doc_embedding)}, Irrelevant Docs = {len(training_triple.irrelevant_doc_embedding)}"
    )
    print(
        f"\tAll are of shaped: {training_triple.relevant_doc_embedding[0].shape}? {all([i.shape == training_triple.relevant_doc_embedding[0].shape for i in training_triple.relevant_doc_embedding])}"
    )
    print(
        f"\tThe query text is {training_dataset_copy[training_dataset_copy.query_id == training_triple.query_id]['query'].values}"
    )
    relevant_text = training_dataset_copy[
        training_dataset_copy.query_id == training_triple[0]
    ]["passages"].values[0]["passage_text"]
    irrelevant_urls_ids = training_dataset_copy[
        training_dataset_copy.query_id == training_triple[0]
    ]["negative_sample_urls"].values[0]
    irrelavant_text = [url_to_doctext_mapping[i] for i in irrelevant_urls_ids]

    # id -> URL ->
    # The relevan
    NL = "\n"
    TAB = "\t"
    print(
        f"\tHere is the relevant docs, { (NL+TAB+TAB).join([str(i)[0:100] + '...' for i in relevant_text])}"
    )
    print(
        f"\t Here are the irrelevant docs, { (NL+TAB+TAB).join([str(i)[0:100] + '...' for i in irrelavant_text])}"
    )

0th triple with query_id: 19699
	#Relevant Docs = 7, Irrelevant Docs = 7
	All are of shaped: (128,)? True
	The query text is ['what is rba']
	Here is the relevant docs, Since 2007, the RBA's outstanding reputation has been affected by the 'Securency' or NPA scandal. Th...
		The Reserve Bank of Australia (RBA) came into being on 14 January 1960 as Australia 's central bank ...
		RBA Recognized with the 2014 Microsoft US Regional Partner of the ... by PR Newswire. Contract Award...
		The inner workings of a rebuildable atomizer are surprisingly simple. The coil inside the RBA is mad...
		Results-Based Accountability® (also known as RBA) is a disciplined way of thinking and taking action...
		Results-Based Accountability® (also known as RBA) is a disciplined way of thinking and taking action...
		RBA uses a data-driven, decision-making process to help communities and organizations get beyond tal...
		vs. NetIQ Identity Manager. Risk-based authentication (RBA) is a method of applying varyi

In [402]:
irrelavant_text

["Overview. As with most legendary Pokemon, Celebi is blessed with great base stats all around. It also has a good and synergetic typing, an awesome ability that lets it act as a status absorber, and a phenomenal movepool. These traits allow Celebi to play many roles, depending on your team's needs. Heatran is possibly the best Celebi partner out there as it has almost flawless defensive synergy with it, with Rock being the only type that the two don't resist together, and also beats many Pokemon that can set up on Celebi or OHKO it, such as Scizor, Forretress, Ferrothorn, and Skarmory.",
 'Calcium Carbonate (Limestone) Calcium carbonate, the chief component of limestone, is a widely used amendment to neutralize soil acidity and to supply calcium (Ca) for plant nutrition. The term “lime” can refer to several products, but for agricultural use it generally refers to ground limestone. ',
 'Health Benefits of Minerals. Below is a list of some of the minerals found in the body, including t

# RNN Definition 

In [472]:
from torch.nn.utils.rnn import pad_sequence
import torch


def custom_collate_fn(batch):
    # Unzip the batch
    query_embs, rel_docs_embs, irrel_docs_embs = zip(*batch)

    # Pad sequences within each batch to the same length
    # Assuming query_embs, rel_docs_embs, and irrel_docs_embs are lists of tensors
    query_embs_padded = pad_sequence(query_embs, batch_first=True, padding_value=0)
    rel_docs_embs_padded = [
        pad_sequence(docs, batch_first=True, padding_value=0)
        for docs in zip(*rel_docs_embs)
    ]
    irrel_docs_embs_padded = [
        pad_sequence(docs, batch_first=True, padding_value=0)
        for docs in zip(*irrel_docs_embs)
    ]

    return query_embs_padded, rel_docs_embs_padded, irrel_docs_embs_padded


class TripletDataset(Dataset):
    def __init__(self, data):
        self.data = data
        # No need to unroll the sequences here

    def __len__(self):
        return len(self.data)  # Returns the total number of triplets

    def __getitem__(self, idx):
        _, query_embedding, relevant_docs_embeddings, irrelevant_docs_embeddings = (
            self.data[idx]
        )
        # No need to iterate through the sequences; return them as they are
        return (
            torch.tensor(query_embedding),
            torch.tensor(relevant_docs_embeddings),
            torch.tensor(irrelevant_docs_embeddings),
        )

In [475]:
training_triplet_dataset = TripletDataset(training_triples)
test_triplet_dataset = TripletDataset(testing_triples)
validate_triplet_dataset = TripletDataset(validation_triples)

In [471]:
import torch

torch.tensor(training_triples[0][2]).shape

torch.Size([7, 128])

In [485]:
for i in range(3):
    print(
        "Query is at index 1 and has Embedding dimension", len(training_triples[i][1])
    )
    print(
        "# Relevant document is len=", len(training_triples[i].relevant_doc_embedding)
    )
    print(
        "Each relevant doc embedding is len=",
        len(training_triples[i].relevant_doc_embedding[0]),
    )
    print(
        "# Irrelvant docs are the same for each relevant doc= ",
        len(training_triples[i].irrelevant_doc_embedding),
    )
    print(
        "# these are too having embedding dimension=",
        len(training_triples[0].irrelevant_doc_embedding[0]),
    )

Query is at index 1 and has Embedding dimension 128
Relevant document is len= 7
Each relevant doc embedding is len= 128
# Irrelvant docs are the same for each relevant doc=  7
# these are too having embedding dimension= 128
Query is at index 1 and has Embedding dimension 128
Relevant document is len= 6
Each relevant doc embedding is len= 128
# Irrelvant docs are the same for each relevant doc=  6
# these are too having embedding dimension= 128
Query is at index 1 and has Embedding dimension 128
Relevant document is len= 6
Each relevant doc embedding is len= 128
# Irrelvant docs are the same for each relevant doc=  6
# these are too having embedding dimension= 128


In [480]:
training_dataloader = DataLoader(
    training_triplet_dataset, batch_size=20, shuffle=False, collate_fn=custom_collate_fn
)

In [481]:
for i, (q, rel, irrel) in enumerate(training_dataloader):
    print(q.shape)
    print(print(len(rel[0]), rel[0].shape))
    print(irrel[0].shape)

    break

torch.Size([20, 128])
20 torch.Size([20, 128])
None
torch.Size([20, 128])


In [ ]:
# LSTM and

In [443]:
# Feed the data through two tower RNN, simple RNNs to
# start. No LSTM or GRUs with a margin and triplet loss.
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import numpy as np


class RNNTower(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super(RNNTower, self).__init__()
        self.rnn = nn.RNN(
            input_size=input_size, hidden_size=hidden_size, batch_first=True
        )
        self.fc = nn.Linear(hidden_size, output_size)

    def forward(self, x):
        # Assuming x is of shape (batch, seq_len, input_size)
        print(x.shape)
        _, hidden = self.rnn(x)
        # hidden is of shape (num_layers * num_directions, batch, hidden_size)
        # For the simple RNN, num_layers * num_directions = 1
        print(hidden.shape)
        hidden = hidden.squeeze(0)  # Now: (batch, hidden_size)
        print(hidden.shape)
        output = self.fc(hidden)
        return output


def triplet_loss(anchor, positive, negative, margin=1.0):
    print(anchor.shape, positive.shape, negative.shape)
    distance_positive = (anchor - positive).pow(2).sum(1)
    distance_negative = (anchor - negative).pow(2).sum(1)
    losses = torch.relu(distance_positive - distance_negative + margin)
    return losses.mean()

In [444]:
# Assuming D is the dimensionality of your embeddings
input_size = EMBEDDING_SIZE  # Dimensionality of the input
hidden_size = 256  # Size of the RNN's hidden layer
output_size = 64  # Size of the final outout

query_rnn = RNNTower(input_size, hidden_size, output_size)
doc_rnn = RNNTower(input_size, hidden_size, output_size)

### NEXT: Training loop for the rnn



In [445]:
optimizer = optim.Adam(
    list(query_rnn.parameters()) + list(doc_rnn.parameters()), lr=0.001
)

In [446]:
for epoch in range(1):
    for query_emb, pos_doc_emb, neg_doc_emb in training_dataloader:
        optimizer.zero_grad()

        # Assuming embeddings are of shape (batch_size, seq_len, input_size)
        # You might need to adjust the shape based on how your data is structured
        query_output = query_rnn(query_emb)
        pos_doc_output = doc_rnn(pos_doc_emb)
        neg_doc_output = doc_rnn(neg_doc_emb)

        loss = triplet_loss(query_output, pos_doc_output, neg_doc_output)
        loss.backward()
        optimizer.step()

    print(f"Epoch {epoch+1}, Loss: {loss.item()}")

torch.Size([12, 128])
torch.Size([1, 256])
torch.Size([256])
torch.Size([12, 128])
torch.Size([1, 256])
torch.Size([256])
torch.Size([12, 128])
torch.Size([1, 256])
torch.Size([256])
torch.Size([64]) torch.Size([64]) torch.Size([64])


IndexError: Dimension out of range (expected to be in range of [-1, 0], but got 1)

In [433]:
(query_output - pos_doc_output).pow(2)

tensor([2.0029e-01, 1.5726e-01, 9.5655e-02, 2.4245e-02, 4.0921e-02, 3.1036e-02,
        2.5124e-02, 3.4916e-01, 8.3306e-02, 9.9364e-02, 1.7097e-01, 1.5423e-01,
        3.3385e-02, 1.1975e-02, 3.9449e-03, 3.1547e-02, 3.8071e-02, 7.3160e-03,
        1.0696e-01, 1.0372e-02, 2.1252e-01, 3.5784e-02, 4.9183e-01, 6.0580e-02,
        2.9841e-02, 1.3620e-01, 3.1525e-01, 1.9255e-02, 4.8204e-02, 4.2308e-02,
        4.0516e-02, 1.9204e-01, 8.7218e-02, 4.2212e-02, 2.8077e-02, 8.6364e-04,
        8.6861e-02, 1.0265e-03, 4.2759e-02, 1.7970e-02, 8.3220e-02, 2.5751e-02,
        2.7908e-01, 5.2570e-02, 3.4769e-02, 9.2672e-02, 1.2490e-01, 4.5189e-04,
        7.1303e-03, 2.9351e-01, 9.6823e-02, 1.0253e-02, 5.6388e-02, 1.4068e-04,
        1.4050e-04, 2.7060e-01, 9.5076e-02, 4.2174e-05, 2.5622e-01, 3.4718e-02,
        1.1543e-04, 5.9856e-02, 4.7871e-02, 5.9104e-04],
       grad_fn=<PowBackward0>)

In [524]:
inputs = [torch.randn(1, 4) for _ in range(5)]

In [514]:
for i in inputs:
    print(".")
    print(i)
    print(i.view(1, 1, -1))

.
tensor([[1.2411, 0.6259, 1.1159]])
tensor([[[1.2411, 0.6259, 1.1159]]])
.
tensor([[0.6223, 1.2001, 0.7883]])
tensor([[[0.6223, 1.2001, 0.7883]]])
.
tensor([[ 0.6911, -1.6744,  0.8264]])
tensor([[[ 0.6911, -1.6744,  0.8264]]])
.
tensor([[-0.0656, -1.2043,  0.6886]])
tensor([[[-0.0656, -1.2043,  0.6886]]])
.
tensor([[0.3123, 0.6284, 0.6254]])
tensor([[[0.3123, 0.6284, 0.6254]]])


In [515]:
inputs = torch.cat(inputs).view(len(inputs), 1, -1)

In [525]:
torch.cat(inputs).view(len(inputs), 1, -1)
# [
#  [[1, 2, 3]],
#     [[4, 5, 6]],
#     [[7, 8, 9]],
#     [[10, 11, 12]],
#     [[13, 14, 15]]
# ]

tensor([[[-0.8364, -0.7349,  0.2562,  1.7806]],

        [[-0.1919, -1.2498,  0.9293, -0.8176]],

        [[-0.1253, -1.9675, -0.9701,  0.7632]],

        [[-0.1750,  0.2369,  0.0190, -0.4398]],

        [[-1.0326, -0.0065,  1.7244, -0.4539]]])

In [526]:
e = nn.Embedding(5, 10)

In [531]:
e(torch.tensor([1, 2, 3])).shape  # 3*10, 3 tokens

torch.Size([3, 10])